In [198]:
import dask.dataframe as dd
import dask
from dask.distributed import Client, TimeoutError, LocalCluster, progress
from dask.diagnostics import ProgressBar
import pandas as pd
import hvplot.dask
import hvplot.pandas
import re
from textblob import TextBlob
from langdetect import detect
from scipy.stats import f_oneway, ttest_ind
from bokeh.models.formatters import DatetimeTickFormatter

#code found on stack exchange
#makes a new client if one is not running, runs 6 workers
try:
    client = Client('tcp://localhost:8787', timeout='5s')
except OSError or TimeoutError:
    cluster = LocalCluster(scheduler_port=8787, n_workers=6, ip='localhost')
    client = Client(cluster)


client


<Client: 'tcp://127.0.0.1:8787' processes=6 threads=12, memory=16.00 GiB>

In [199]:
# reference variables
keyWords = [
    'mental health', 
    'depression',
    'depressed', 
    'anxiety',
    'anxious', 
    'ADHD', 
    'attention deficit',
    'OCD',
    'obsessive-compulsive disorder',
    'PTSD',
    'posttraumatic stress disorder',
    'trauma',
    'bipolar',
    'autism',
    'schizophrenia',
    'eating disorder',
    'anorexia',
    'bulimia',
    'psychosis',
    'psychologist',
    'psychotherapist',
    'psychiatrist',
    'psychotherapy',
    'depressant'
]

keyCats = {
    'gen' : ['mental health', 'trauma', 'psychosis', 'psychologist', 'psychotherapist', 'psychiatrist', 'psychotherapy'],
    'dep' : ['depression', 'depressed', 'depressant'],
    'anx' : ['anxiety', 'anxious'],
    'ocd' : ['OCD', 'obsessive-compulsive disorder'],
    'adhd' : ['ADHD', 'attention deficit'],
    'ptsd' : ['PTSD', 'posttraumatic stress disorder'],
    'bip' : ['bipolar'],
    'asd' : ['autism'],
    'schiz' : ['schizophrenia'],
    'ed' : ['eating disorder', 'anorexia', 'bulimia']
}

abbrCats = {
    'gen' : 'General',
    'dep' : 'Depression',
    'anx' : 'Anxiety',
    'ocd' : 'OCD',
    'adhd' : 'ADHD',
    'ptsd' : 'PTSD',
    'bip' : 'Bipolar',
    'asd' : 'Autism',
    'schiz' : 'Schizophrenia',
    'ed' : 'Eating Disorder'
}

sentCols = ['subjectivity', 'polarity']

In [200]:
#contains counts of tweets for every hour
countdf = dd.read_csv(urlpath='mental_health_tweet_count.csv', sep='\t', lineterminator='\n')

#try reading in the preprocessed tweets
# if the csv does not exist, create it
try:
    # read the preprocessed csv
    tweetdf = dd.read_parquet(path='mental_health_tweets_preprocessed_en.parquet')
except:
    #contains tweets (and time of the tweet) that contain mental health key word
    #polled every 2 minutes
    tweetdf = dd.read_csv(urlpath='mental_health_tweets.csv', sep='\t', lineterminator='\n')
    
    #pre process
    # preprocess instructions found at https://youtu.be/ujId4ipkBio
    def preprocess(text):
        # remove mentions
        text = re.sub('@[a-zA-Z0-9]+', '', text)
        
        #  remove hashtag symbol
        text = re.sub('#', '', text)
        
        # remove retweet identifier
        text = re.sub('RT[\s]+', '', text)
        
        # remove links
        text = re.sub('https?:\/\/\S+', '', text)


        text = re.sub('[^a-zA-Z\\s]', '', text)

        text = text.strip()

        return text

    tweetdf['text'] = tweetdf['tweet'].map(preprocess)

    # remove any tweets that are either empty, or not in english
    tweetdf = tweetdf.loc[tweetdf['text'] != '']
    tweetdf['language'] = tweetdf['text'].map(lambda x: detect(x))
    tweetdf = tweetdf.loc[tweetdf['language'] == 'en']

    # drop original tweet and language columns
    tweetdf = tweetdf.drop(columns=['tweet', 'language'])

    #write pre processed text to parquet (dask recommends parquet)
    dd.to_parquet(tweetdf, path='mental_health_tweets_preprocessed_en.parquet')
    
    

In [201]:

#specify column types
tweetdf['time'] = dd.to_datetime(tweetdf['time'])
countdf['timestamp'] = dd.to_datetime(countdf['timestamp'])

#separate the components of the time for grouping
tweetdf['year'] = tweetdf['time'].dt.year
tweetdf['month'] = tweetdf['time'].dt.month
tweetdf['day'] = tweetdf['time'].dt.day
tweetdf['hour'] = tweetdf['time'].dt.hour
tweetdf['minute'] = tweetdf['time'].dt.minute

# only keep tweets from november 18th-30th
tweetdf = tweetdf.loc[tweetdf['month'] == 11]
tweetdf = tweetdf.loc[tweetdf['day'] > 17]

countdf['year'] = countdf['timestamp'].dt.year
countdf['month'] = countdf['timestamp'].dt.month
countdf['day'] = countdf['timestamp'].dt.day
countdf['hour'] = countdf['timestamp'].dt.hour

# only keep counts from november 18th-30th
countdf = countdf.loc[countdf['month'] == 11]
countdf = countdf.loc[countdf['day'] > 17]

In [202]:
#sort the dataframes based on ascening tme
countdf = countdf.sort_values(ascending=True, by=['month', 'day', 'hour'])
tweetdf = tweetdf.sort_values(ascending=True, by=['month', 'day', 'hour', 'minute'])

#make a display string for the x axis of the graph
countdf['ts_display'] = countdf['month'].astype('str') + '/' + countdf['day'].astype('str') + '\t' + countdf['hour'].astype('str') + ':00'

numHours = len(tweetdf.groupby(['month', 'day', 'hour']).count().compute())
numHours /= 8
numHours = int(numHours)
print(numHours)
formatter = DatetimeTickFormatter(months='%m/%d %H:00', days='%m/%d %H:00', hours='%m/%d %H:00')
hvplot.save(countdf.hvplot(x='timestamp', y='count', title='Tweet Count', xlabel='date', ylabel='# tweets', xformatter=formatter).opts(xrotation=90, fontscale=0.75, xticks=numHours),\
    'figures/tweets-over-time.png')

39


In [203]:
#determine key words in tweet
def containsKey(text, key):
    return text.str.contains(key)

# create a new column for each key, with values of whether or not
# the tweet in that row contains that key
for i in keyWords:
    tweetdf[i] = tweetdf['text'].map_partitions(containsKey, i)


In [204]:
# given key k and dataframe df, find the number of times k appears in the df
def countKeys(k, df):
    return df[k].sum()


futures = []

for k in keyWords:
    futures.append(client.submit(countKeys, k, tweetdf))

keyCount = client.gather(futures)

keyCount = dask.compute(*keyCount)


keyCountdf = pd.DataFrame(columns=['key', 'count'], data=list(zip(keyWords, keyCount)))

keyCountdf = keyCountdf.sort_values(by='count', axis=0)

hvplot.save(keyCountdf.hvplot.bar(y='count', x='key', title='Key Count', xlabel='key', ylabel='# tweets').opts(fontscale=0.75, invert_axes=True),\
    'figures/individual-key-count-bar.png')

Task exception was never retrieved
future: <Task finished name='Task-68403' coro=<Client._gather.<locals>.wait() done, defined at /Users/derek/opt/anaconda3/lib/python3.9/site-packages/distributed/client.py:2002> exception=AllExit()>
Traceback (most recent call last):
  File "/Users/derek/opt/anaconda3/lib/python3.9/site-packages/distributed/client.py", line 2011, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-68404' coro=<Client._gather.<locals>.wait() done, defined at /Users/derek/opt/anaconda3/lib/python3.9/site-packages/distributed/client.py:2002> exception=AllExit()>
Traceback (most recent call last):
  File "/Users/derek/opt/anaconda3/lib/python3.9/site-packages/distributed/client.py", line 2011, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-68405' coro=<Client._gather.<locals>.wait() done, defined at /Users/derek/opt/anaconda3

In [205]:
#count tweets each keyword appears in
#if a tweet contains multiple keywords, then they are added 
#to the result df as |word1|word2| etc.

def keysInTweet(df):
    keys = ''
    for k in keyWords:
        if df[k]:
            keys = keys + '|' + k
    keys = keys + '|'
    return keys

tweetdf['keys'] = tweetdf.apply(keysInTweet, axis=1, meta=('keys', 'object'))

# drop tweets with no keys
tweetdf = tweetdf.loc[tweetdf['keys'] != '|']

# convert the value counts to its own df
multiKeyCount = tweetdf['keys'].value_counts().to_frame()
# copy the key column (which actually holds counts), rename it and then drop the original
multiKeyCount['count'] = multiKeyCount['keys']
multiKeyCount = multiKeyCount.drop(labels='keys', axis=1)

multiKetCount = multiKeyCount.sort_values(by='count', ascending=False)

multiKeyCount = multiKeyCount.head(10)
hvplot.save(multiKeyCount.head(10).hvplot.bar(y='count', use_index=True, title='Top 10 Topic Combinations', xlabel='topic combinations', ylabel='# tweets').opts(xrotation=90),\
    'figures/key-groups-count-bar.png')

In [206]:
# determine whether each tweet has keywords related to each category
def getCategories(row, keyList):
    for k in keyList:
        if row[k]:
            return True
        
    return False


# create a new column for each category
for c in keyCats.keys():
    tweetdf[c] = tweetdf.apply(getCategories, args=([keyCats[c]]), axis=1, meta=(c, 'boolean'))

# drop the original keyword columns
tweetdf = tweetdf.drop(columns=keyWords)


In [207]:


def getCatCount(c, df):
    return df[c].sum()

#client submit and gather implementation modified from stackexchange
futures = []
for c in keyCats.keys():
    futures.append(client.submit(getCatCount, c, tweetdf))

catCount = client.gather(futures)

catCount = dask.compute(*catCount)
catCountdf = pd.DataFrame(columns=['category', 'count'], data=list(zip(keyCats.keys(), catCount)))

catCountdf = catCountdf.sort_values(by='count', axis=0, ascending=False)

catCountdf['category'] = catCountdf['category'].map(lambda x: abbrCats[x])

hvplot.save(catCountdf.hvplot.bar(x='category', y='count', xlabel='topic', ylabel='# tweets', title='Tweets per Topic').opts(xrotation=90),\
    'figures/topic-count-bar.png')


In [208]:
# get subjectivity of each tweet [0, 1]
def compSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

# get polarity of tweet [-1, 1]
def compPolarity(text):
    return TextBlob(text).sentiment.polarity

# add subjectivity and polarity as new columns
tweetdf['subjectivity'] = tweetdf['text'].map(compSubjectivity)
tweetdf['polarity'] = tweetdf['text'].map(compPolarity)

# speeds up computation of mean and std of sub and pol for each cat
tweetdf.compute()
tweetdf = tweetdf.persist()

tweetdf

,time,text,year,month,day,hour,minute,keys,gen,dep,anx,ocd,adhd,ptsd,bip,asd,schiz,ed,subjectivity,polarity
npartitions=1,,,,,,,,,,,,,,,,,,,,
,datetime64[ns],object,int64,int64,int64,int64,int64,object,boolean,boolean,boolean,boolean,boolean,boolean,boolean,boolean,boolean,boolean,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [209]:
# can store results in a pandas dataframe since we know it will fit in memory
mlres = pd.DataFrame()

# for each category, make a series of sub and pol mean and std
# then add it to the output pandas df
for c in keyCats.keys():
    tmp = tweetdf.loc[tweetdf[c]]
    data = (tmp['subjectivity'].mean().compute(),\
            tmp['polarity'].mean().compute(),\
            tmp['subjectivity'].std().compute(),\
            tmp['polarity'].std().compute()
            )
    ser = pd.Series(data=data, index=['subjectivity mean', 'polarity mean', 'subjectivity std', 'polarity std'], name=c)
    mlres = pd.concat([mlres, ser], axis=1)

mlres = mlres.rename(abbrCats)


In [220]:
mlres

,gen,dep,anx,ocd,adhd,ptsd,bip,asd,schiz,ed
subjectivity mean,0.397486,0.418807,0.446457,0.401190,0.389963,0.394128,0.411973,0.381295,0.375805,0.404271
polarity mean,0.024546,0.058445,0.000866,0.042355,0.066950,0.046806,0.020703,0.051647,0.012839,0.024946
subjectivity std,0.222610,0.292910,0.342674,0.303571,0.283639,0.285857,0.303990,0.298897,0.295384,0.303036
polarity std,0.221095,0.283889,0.245240,0.259828,0.246970,0.250086,0.276940,0.250761,0.261636,0.278271


In [210]:
hvplot.save(mlres.drop(index=['subjectivity std', 'polarity std', 'polarity mean']).hvplot.bar(use_index=True, title='Subjectivity Means', xlabel='Topic', ylabel='Subjectivity', ylim=(0, 1)),\
    'figures/subjectivity-mean-bar.png')

In [211]:
hvplot.save(mlres.drop(index=['subjectivity std', 'polarity std', 'subjectivity mean']).hvplot.bar(use_index=True, title='Polarity Means', xlabel='Topic', ylabel='Polarity', ylim=(-1, 1)),\
    'figures/polarity-mean-bar.png')

In [217]:
# print the tweets that fall on each end (and middle for polarity) of the sentiment analysis
polDF = tweetdf.groupby('polarity').min().compute()
subjDF = tweetdf.groupby('subjectivity').min().compute()


print(f"Polarity = -1:\n{polDF['text'][-1]}\n")
print(f"Polarity = 0:\n{polDF['text'][0]}\n")
print(f"Polarity = 1:\n{polDF['text'][1]}\n")
print(f"Subjectivity = 0:\n{subjDF['text'][0]}\n")
print(f"Subjectivity = 0.5:\n{subjDF['text'][0.5]}\n")
print(f"Subjectivity = 1:\n{subjDF['text'][1]}")

Polarity = -1:
A conversion ban means you can keep not doing your job amp let children selfdiagnose while you nod along I can see why a horrible person would continue to collect pay for not doing anything a therapist should do like exploratory psychotherapy to find the causes of the trauma

Polarity = 0:
A  Increase education and awareness about ACEs adverse childhood experiences in order to help youth and communities understand the way early adversity impacts health and mental health endyouthhomelessness TNOYSTownHall

Polarity = 1:
A portray of a depressed person from a depressed person magnificent

Subjectivity = 0:
A  is truly not for me I feel so depressed

Subjectivity = 0.5:
A  Victorian parliamentary enquiry into lowering the driving age to  found doing so would increase the instances of road trauma which would put even more stress on hospitals and ambulances  so of course the Libs would be all for it

Subjectivity = 1:
A Maryland couple drove two hours to adopt the timidlookin

In [213]:
# counts the number of tweets that fall into each sentiment analysis category
numMoreObj = len(tweetdf.loc[tweetdf['subjectivity'] < 0.5].compute())
numMoreSubj = len(tweetdf.loc[tweetdf['subjectivity'] > 0.5].compute())
numEquallySubjObj = len(tweetdf.loc[tweetdf['subjectivity'] == 0].compute())

numPos = len(tweetdf.loc[tweetdf['polarity'] > 0].compute())
numNeg = len(tweetdf.loc[tweetdf['polarity'] < 0].compute())
numNeut = len(tweetdf.loc[tweetdf['polarity'] == 0].compute())

print(f"Objective leaning: {numMoreObj}")
print(f"Subjective leaning: {numMoreSubj}")
print(f"Equally subjective and objective: {numEquallySubjObj}")
print(f"Positive: {numPos}")
print(f"Negative: {numNeg}")
print(f"Neutral: {numNeut}")

Objective leaning: 71080
Subjective leaning: 47452
Equally subjective and objective: 23683
Positive: 51196
Negative: 43399
Neutral: 28535


In [214]:
alpha = 0.001
sigOpts = ['significant', 'not significant']

# One way ANOVA on column
def owANOVA(col):
    catArrs = []
    for c in keyCats.keys():
        tmp = tweetdf.loc[tweetdf[c]]
        tmp = tmp[col]
        tmp = tmp.to_dask_array()
        catArrs.append(tmp)

    return f_oneway(*catArrs)

owaSubj = owANOVA('subjectivity')
owaPol = owANOVA('polarity')


def owaIsSig(pval, alpha):
    if pval < alpha:
        return sigOpts[0]
    else:
        return sigOpts[1]

owaStr = f"Subjectivity: {owaIsSig(owaSubj.pvalue, alpha)}, {owaSubj[1]}\n"
owaStr = owaStr + f"Polarity: {owaIsSig(owaPol.pvalue, alpha)}, {owaPol[1]}"

print(owaStr)

Subjectivity: significant, 7.612887176705211e-161
Polarity: significant, 2.470843946377159e-226


In [215]:
# test on each pair of categories
def tTest(col, a, b):
    a = tweetdf.loc[tweetdf[a]]
    a = a[col]
    a = a.to_dask_array()
    b = tweetdf.loc[tweetdf[b]]
    b = b[col]
    b = b.to_dask_array()
    return ttest_ind(a=a, b=b, equal_var=False)
    
def allTTests(col):
    out = []
    keys = list(keyCats.keys())
    for x in range(len(keys)):
        for y in range(x+1, len(keys)):
            out.append((keys[x], keys[y], tTest(col, keys[x], keys[y])))
    return out

subjT = allTTests('subjectivity')
polT = allTTests('polarity')

def statSigT(res, alpha):
    out = []
    for i in res:
        sig = i[2][1] < alpha
        isSig = (abbrCats[i[0]], abbrCats[i[1]], sig)
        out.append(isSig)
    return out

subjSig = statSigT(subjT, alpha)
polSig = statSigT(polT, alpha)

def sortRes(lst):
    out = {
        sigOpts[0] : [],
        sigOpts[1] : []
    }

    sig = None
    for i in lst:
        if i[2]:
            sig = sigOpts[0]
        else:
            sig = sigOpts[1]
        out[sig].append(f"{i[0]} and {i[1]}")
    return out

subjSig = sortRes(subjSig)
polSig = sortRes(polSig)


def sigStr(lst, title):
    outStr = '********************\n'
    outStr = outStr + f"{title.upper()}:\n"

    for i in sigOpts:
        outStr = outStr + '\n--------------------\n'
        outStr = outStr + i + ':'
        outStr = outStr + '\n--------------------\n'
        for j in lst[i]:
            outStr = outStr + j + '\n'

    outStr = outStr + '********************\n'
    return outStr

tStr = sigStr(subjSig, 'subjectivity') + sigStr(polSig, 'polarity')

print(tStr)

********************
SUBJECTIVITY:

--------------------
significant:
--------------------
General and Depression
General and Anxiety
General and Autism
Depression and Anxiety
Depression and ADHD
Depression and PTSD
Depression and Autism
Depression and Schizophrenia
Anxiety and OCD
Anxiety and ADHD
Anxiety and PTSD
Anxiety and Bipolar
Anxiety and Autism
Anxiety and Schizophrenia
Anxiety and Eating Disorder
Bipolar and Autism

--------------------
not significant:
--------------------
General and OCD
General and ADHD
General and PTSD
General and Bipolar
General and Schizophrenia
General and Eating Disorder
Depression and OCD
Depression and Bipolar
Depression and Eating Disorder
OCD and ADHD
OCD and PTSD
OCD and Bipolar
OCD and Autism
OCD and Schizophrenia
OCD and Eating Disorder
ADHD and PTSD
ADHD and Bipolar
ADHD and Autism
ADHD and Schizophrenia
ADHD and Eating Disorder
PTSD and Bipolar
PTSD and Autism
PTSD and Schizophrenia
PTSD and Eating Disorder
Bipolar and Schizophrenia
Bipolar a

In [216]:
with open('figures/stat_results.txt', 'w') as filehandler:
    filehandler.write(f"One-Way ANOVA Results:\n{owaStr}\n\nT Tests Results:\n{tStr}")